# Example 7: Backtesting a `SignalStrategy` using your own signals data

## Pre-requisites
### 1. Install our SDK locally
This example cannot be run in Colab as it requires an additional CSV file. 
In a command prompt or terminal window:
1. Navigate to where you would like to install our SDK.
1. Enter the command `pip install sigtech`.

### 2. Open this notebook in an IDE

### 3. Enter your API key in the cell below

In [ ]:
# Install our Python SDK
%pip install sigtech 

# Import OS and our Python SDK
import sigtech.api as sig
import os

# Define your API key as a string. Remember to delete it before sharing your notebook with others. Replace 
# <YOUR_API_KEY> with the API key you have generated. e.g. os.environ['SIGTECH_API_KEY'] = 'sig_A1B2C3D4E5f6g7h8i9'
os.environ['SIGTECH_API_KEY'] = '<YOUR_API_KEY>'

### 4. Set up your local environment

In [ ]:
# Import any additional Python libraries you require.
import datetime as dtm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Set any parameters 
plt.rcParams['figure.figsize'] = [16, 8]

### 5. Create a session
After installing our Python SDK, defining your API key, importing any additional Python libraries or functions you require, and setting any default parameters, initialize your session.

In [ ]:
sig.init()

This example is designed to showcase how you can use your own data to generate trading signals and then backtest the performance of a strategy based on those signals in minutes using our curated data and API.

## Our strategy
In this example, we are backtesting the performance of a strategy which trades US 10-Year note futures based on a set of trading signals generated using the FRED API. These signals are available in `example_data/ty_signal.csv`. How the trading signals were generated is not important; remember, you can easily substitute our `ty_signal.csv` file for a CSV file with your own trading signals and backtest the performance of a strategy which uses those signals instead.  


## 1. View the baseline performance of the commodity

In this example, we will be looking at US 10-Year note futures. To plot the historical performance of this commodity we use the `RollingFuturesStrategy` class. 
>**Tip**!\
>You can find the `contract_code` and `contract_sector` for a commodity using our [API data browser](https://sigtechapi.streamlit.app/).

In [ ]:
ty = sig.RollingFutureStrategy(
    contract_code = 'TY',
    contract_sector = 'COMDTY', 
    start_date = dtm.date(2016,1,1)

In [ ]:
ty.history().plot()

## 2. Compare it to the performance of the `SignalStrategy`

Firstly, we pull in data from our CSV file in the `example_data` folder.

In [ ]:
# Replace <path_to_file> with the filepath for where you have installed our SDK.
signal = pd.read_csv('<path_to_file>example_data/ty_signal.csv', index_col = 0)

Next, we must convert the CSV data so that it can be used as the `signal_input` for our `SignalStrategy`.
> A `SignalStrategy` requires a `signal_input`. This is a pandas DataFrame where the column headers are the instrument names and the values are the signals for each of the instruments. These signals can be either a number of units *or* a weight.

In [ ]:
signal.index = pd.to_datetime(signal.index)
signal.columns = [ty.name]

Run the code cell below to see what a dataframe that can be used as a `signal_input` looks like.

In [ ]:
signal.tail()
signal.plot()

Finally, we can create a `SignalStrategy` and view its performance. 

In [ ]:
s = sig.SignalStrategy(
    signal_input = signal,
    currency = 'USD',
    rebalance_frequency = 'EOM', # The strategy will rebalance at the end of each month. See https://learn.sigtech.com/docs/rebalance_frequencies for more information. 
)

In [ ]:
s.history().plot(label = 'Signal Strategy', legend=True)
ty.history().plot(label = 'Baseline', legend=True)
